Note: The codes below are only for testing video 2. To make the code run perfectly for other videos ROI has to be defined for that video.

In [2]:
#Lucas Kanade for velocity 
import numpy as np
import cv2 as cv

cap = cv.VideoCapture("testing_video_2.mp4")

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
# Apply Gaussian blur with a 3x3 kernel and sigma of 0
old_gray = cv.GaussianBlur(old_gray, (3, 3), 0)
mask = np.zeros_like(old_gray)

# Define the ROI coordinates
x, y, w, h = 46, 288, 134, 351

# Set the ROI region in the mask to 1
mask[y:y+h, x:x+w] = 1
p0 = cv.goodFeaturesToTrack(old_gray, mask = mask, **feature_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

# Create a dictionary to store the tracked points and their IDs
points = {}
id_counter = 0
N=0
while(1):
    ret, frame = cap.read()
    if N==53:
        print('Done')
        break

    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # frame_gray = frame_gray[y:y+h, x:x+w]
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Update the ROI coordinates based on the new pixel positions
    x += int(p1.mean(axis=0)[0][0] - p0.mean(axis=0)[0][0])
    y += int(p1.mean(axis=0)[0][1] - p0.mean(axis=0)[0][1])
    # Draw the updated ROI on the current frame
    cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]

    # Draw the tracks and update the points dictionary
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
        
        # Update the tracked points dictionary
        if i not in points:
            points[i] = []
        points[i].append((a, b))

    img = cv.add(frame, mask)
    new_velocity=0
    a=np.array([])
    for i, pts in points.items():
        if len(pts) < 2:
            continue
        prev_point = pts[-2]
        curr_point = pts[-1]
        x1, y1 = prev_point
        x2, y2 = curr_point
        distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        elapsed_time = 1 / cap.get(cv.CAP_PROP_FPS)
        velocity = distance / elapsed_time
        a=np.append(a,velocity)
        if velocity!=0:
            i_current=i 
    if a is not None:
        print("velocity:", np.mean(a))
    
    N+=1
    cv.imshow('frame', img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv.destroyAllWindows()

velocity: nan
velocity: 110.49462994565238
velocity: 117.02106414526095
velocity: 120.28444495605711
velocity: 124.11585928022896
velocity: 126.0700094834579
velocity: 127.11522894966815
velocity: 131.77943322115428
velocity: 134.69206837155056
velocity: 140.19291723363443
velocity: 143.2753645567784
velocity: 148.03362237426137
velocity: 150.33195930518568
velocity: 152.97059711168396
velocity: 157.46632162253107
velocity: 160.02991553539525
velocity: 163.73081606024633
velocity: 163.83194963998966
velocity: 167.7881019917227
velocity: 167.01094477797037
velocity: 169.88756308108
velocity: 169.71457033015864
velocity: 168.12296097916084
velocity: 169.12719581848873
velocity: 168.6462013423953
velocity: 171.29195635816808
velocity: 165.58228083733437
velocity: 160.2079040922893
velocity: 155.284573943635
velocity: 157.33097081985557
velocity: 158.10613770611585
velocity: 152.32379996691353
velocity: 146.15537598860843
velocity: 139.47760274229762
velocity: 130.6050692968438
velocity: 1

The code below can show multiple types of outputs, read the comments properly. Farne back is needed to plot the optical flow and arrows for optical shading. Lucas Kanade is used for velocity calculations.

In [6]:
#Forneback and Lucas Kanade combined for velocity 

import numpy as np
import cv2 as cv

# Read in video file
cap = cv.VideoCapture("testing_video_2.mp4")
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Read in first frame and select ROI around object
ret, frame = cap.read()
# roi = cv.selectROI(frame)
x, y, w, h = 95, 276, 83, 71
# Initialize variables
prev_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
# define a kernel for dilation
# kernel = cv.getStructuringElement(cv.MORPH_RECT, (5,5))
# prev_frame= cv.dilate(prev_frame, kernel, iterations=1)
mask = np.zeros_like(prev_frame)
mask[y:y+h, x:x+w] = 1
p0= cv.goodFeaturesToTrack(prev_frame, mask = mask, **feature_params)
hsv = np.zeros_like(frame)
hsv[..., 1] = 255
num_frames = 0
N=0
# Create a dictionary to store the tracked points and their IDs
points = {}
id_counter = 0
# Loop over video frames
while True:
    # Read in next frame
    ret, frame = cap.read()
    if N==53:
        break
    
    # Convert frames to grayscale and calculate optical flow
    curr_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    p1, st, err = cv.calcOpticalFlowPyrLK(prev_frame, curr_frame, p0, None, **lk_params)
    # Update the ROI coordinates based on the new pixel positions
    x += int(p1.mean(axis=0)[0][0] - p0.mean(axis=0)[0][0])
    y += int(p1.mean(axis=0)[0][1] - p0.mean(axis=0)[0][1])
    # Draw the updated ROI on the current frame
    
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]

    flow = cv.calcOpticalFlowFarneback(prev_frame, curr_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        #Uncomment to see the feature points on the car being tracked
        # cv.circle(frame, (int(a), int(b)), 5, (255,0,0), -1)
        if i not in points:
            points[i] = []
        points[i].append((a, b))
    # Compute the velocity of each car and print its ID and velocity
    V_list=np.array([])
    for i, pts in points.items():
        if len(pts) < 2:
            continue
        prev_point = pts[-2]
        curr_point = pts[-1]
        x1, y1 = prev_point
        x2, y2 = curr_point
        distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        elapsed_time = 1 / cap.get(cv.CAP_PROP_FPS)
        velocity = distance / elapsed_time
        if velocity!=0:
            V_list=np.append(V_list,velocity)
            # print("Object id",i,"velocity:", velocity)
    print("velocity:", np.mean(V_list))
    # # Draw optical flow lines on frame
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    cv.rectangle(bgr, (x, y), (x + w, y + h), (255, 0, 0), 2)
    #Uncomment the next line to watch optical flow
    cv.imshow("frame", bgr)
    #Uncomment the next line to see the feature points being tracked on the car
    # cv.imshow("frame", frame)
    # Create a grid of points to display arrows
    step = 20
    h, w = frame.shape[:2]
    q, u = np.mgrid[int(step / 2):h:step, int(step / 2):w:step].reshape(2, -1).astype(int)
    # Calculate coordinates of end points of arrows
    fx, fy = flow[q, u].T
    e, f = np.round(u + fx).astype(int), np.round(q + fy).astype(int)
    # Draw arrows on the image
    for i, j, k, l in zip(u, q, e, f):
        cv.arrowedLine(frame, (i, j), (k+10, l+10), (0, 255, 0), 1)
    N+=1
    p0=p1
    #Use the next line to see the optical shade
    # cv.imshow("frame", frame)
    key = cv.waitKey(1)
    if key == 27:
            break
    
    # Update variables for next iteration
    prev_frame = curr_frame
    num_frames += 1
cv.destroyAllWindows()


velocity: nan
velocity: 110.63130109878055
velocity: 117.49972922258752
velocity: 120.37222184209087
velocity: 124.85247721051415
velocity: 125.66582373239784
velocity: 127.58454197246807
velocity: 132.36574468780665
velocity: 135.67540551808858
velocity: 140.35717563970738
velocity: 144.07501799548695
velocity: 149.08086806222306
velocity: 150.78822761606258
velocity: 153.38399845707264
velocity: 158.07407498442458
velocity: 160.0063054773971
velocity: 164.87061503963218
velocity: 164.83411110773798
velocity: 169.48232628338988
velocity: 169.13346373655338
velocity: 169.49656462931216
velocity: 171.98878937140762
velocity: 169.93736572176832
velocity: 171.26935191174826
velocity: 172.89453455500131
velocity: 173.8380121936434
velocity: 168.96052548547593
velocity: 161.25209739589627
velocity: 158.23305501050416
velocity: 160.44265348998238
velocity: 162.00481241669843
velocity: 156.3354643635429
velocity: 148.62943885810088
velocity: 144.20751625906223
velocity: 134.22282839526991
vel